In [ ]:
%%bash
set -euxo pipefail
export DEBIAN_FRONTEND=noninteractive

echo "[STEP 1/6] base packages $(date -u)"
apt-get update
apt-get install -y --no-install-recommends \
  ca-certificates git curl unzip zip openjdk-17-jdk-headless

echo "[DONE] step 1/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 2/6] android cmdline-tools $(date -u)"
mkdir -p "$ANDROID_SDK_ROOT/cmdline-tools"
if [ ! -x "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" ]; then
  curl -fL --retry 5 --retry-delay 5 \
    https://dl.google.com/android/repository/commandlinetools-linux-11076708_latest.zip \
    -o /tmp/android-cmdline-tools.zip
  unzip -q /tmp/android-cmdline-tools.zip -d "$ANDROID_SDK_ROOT/cmdline-tools"
  rm -rf "$ANDROID_SDK_ROOT/cmdline-tools/latest"
  mv "$ANDROID_SDK_ROOT/cmdline-tools/cmdline-tools" "$ANDROID_SDK_ROOT/cmdline-tools/latest"
fi

echo "[DONE] step 2/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 3/6] accept sdk licenses $(date -u)"
set +o pipefail
timeout 900 bash -lc 'yes | /content/android-sdk/cmdline-tools/latest/bin/sdkmanager --sdk_root=/content/android-sdk --licenses --verbose' \
  > /tmp/sdk-licenses.log 2>&1
licenses_rc=$?
set -o pipefail
tail -n 120 /tmp/sdk-licenses.log || true
if [ "$licenses_rc" -eq 124 ]; then
  echo "sdkmanager --licenses timeout (15 min)"
  exit 124
fi
if [ "$licenses_rc" -ne 0 ]; then
  echo "sdkmanager --licenses failed: $licenses_rc"
  exit "$licenses_rc"
fi

echo "[DONE] step 3/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
export ANDROID_SDK_ROOT=/content/android-sdk

echo "[STEP 4/6] install sdk packages $(date -u)"
set +o pipefail
yes | "$ANDROID_SDK_ROOT/cmdline-tools/latest/bin/sdkmanager" --sdk_root="$ANDROID_SDK_ROOT" \
  "platform-tools" "platforms;android-34" "build-tools;34.0.0"
sdk_rc=${PIPESTATUS[1]}
set -o pipefail
if [ "$sdk_rc" -ne 0 ]; then
  echo "sdkmanager packages failed: $sdk_rc"
  exit "$sdk_rc"
fi

echo "[DONE] step 4/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail

echo "[STEP 5/6] write env $(date -u)"
cat > /content/tgwebmobile_env.sh <<'ENVEOF'
export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
export ANDROID_SDK_ROOT=/content/android-sdk
export ANDROID_HOME=$ANDROID_SDK_ROOT
export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
ENVEOF

echo "[DONE] step 5/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[STEP 6/6] clone project + prepare local.properties $(date -u)"
cd /content
if [ ! -d tgwebmobile ]; then
  git clone https://github.com/Perdonus/tgwebmobile.git
fi

cd /content/tgwebmobile
git fetch origin main
git reset --hard origin/main

cat > local.properties <<EOF2
sdk.dir=/content/android-sdk
EOF2

# Optional files (upload manually to /content before this step):
# /content/google-services.json, /content/keystore.jks, /content/keystore.properties
if [ -f /content/google-services.json ]; then cp /content/google-services.json app/google-services.json; fi
if [ -f /content/keystore.jks ]; then cp /content/keystore.jks app/keystore.jks; fi
if [ -f /content/keystore.properties ]; then cp /content/keystore.properties keystore.properties; fi

chmod +x gradlew
echo "[DONE] step 6/6 $(date -u)"


In [ ]:
%%bash
set -euxo pipefail

if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

cd /content
if [ ! -d tgwebmobile ]; then
  git clone -b main https://github.com/Perdonus/tgwebmobile.git
fi

cd /content/tgwebmobile
git fetch origin main
git reset --hard origin/main
git --no-pager log --oneline -n 5


In [ ]:
# STEP 6.5/6: upload Firebase files from local PC (after git sync)
from google.colab import files
import json
import os
import shutil

REPO = '/content/tgwebmobile'
if not os.path.isdir(REPO):
    raise RuntimeError(f'Repository not found: {REPO}. Run git clone/sync cell first.')

os.makedirs(f'{REPO}/app', exist_ok=True)
os.makedirs(f'{REPO}/firebase', exist_ok=True)

print('Upload exactly 2 files:')
print('1) google-services.json')
print('2) Firebase Admin SDK service account JSON')
uploaded = files.upload()
if len(uploaded) < 2:
    raise RuntimeError('Please upload both files (google-services.json + service account JSON).')

google_src = None
admin_src = None

for name, content in uploaded.items():
    path = f'/content/{name}'
    with open(path, 'wb') as f:
        f.write(content)

    lower = name.lower()
    if lower == 'google-services.json' or 'google-services' in lower:
        google_src = path
    if 'adminsdk' in lower or 'service-account' in lower or 'firebase-admin' in lower:
        admin_src = path

# Fallback detection by JSON shape.
if not google_src or not admin_src:
    for name in uploaded:
        path = f'/content/{name}'
        try:
            with open(path, 'r', encoding='utf-8') as f:
                root = json.load(f)
        except Exception:
            continue

        if not google_src and isinstance(root, dict) and 'project_info' in root and 'client' in root:
            google_src = path
        if not admin_src and isinstance(root, dict) and root.get('type') == 'service_account' and 'private_key' in root:
            admin_src = path

if not google_src:
    raise RuntimeError('Cannot detect google-services.json in uploaded files.')
if not admin_src:
    raise RuntimeError('Cannot detect Firebase service-account JSON in uploaded files.')

google_dst = f'{REPO}/app/google-services.json'
admin_dst = f'{REPO}/firebase/firebase-adminsdk.json'

shutil.copy2(google_src, google_dst)
shutil.copy2(admin_src, admin_dst)

with open(admin_dst, 'r', encoding='utf-8') as f:
    admin = json.load(f)
project_id = admin.get('project_id', '')

package_name = ''
try:
    with open(google_dst, 'r', encoding='utf-8') as f:
        g = json.load(f)
    client = (g.get('client') or [])[0]
    package_name = client.get('client_info', {}).get('android_client_info', {}).get('package_name', '')
except Exception:
    pass

env_path = '/content/tgwebmobile_push_env.sh'
with open(env_path, 'w', encoding='utf-8') as f:
    f.write(f'export FCM_PROJECT_ID="{project_id}"\n')
    f.write(f'export FCM_SERVICE_ACCOUNT_JSON="{admin_dst}"\n')
    f.write('export PUSH_BIND_HOST="192.168.1.109"\n')
    f.write('export PUSH_PORT="8081"\n')
    f.write('export PUSH_BASE_PATH="/flygram/push"\n')
    f.write('export PUSH_SHARED_SECRET="flygram_push_2026"\n')

print('\nCopied files:')
print(' -', google_dst)
print(' -', admin_dst)
print('Project ID:', project_id or '(not found)')
print('Android package in google-services:', package_name or '(not found)')
print('Env file created:', env_path)


In [ ]:
%%bash
set -euxo pipefail
if [ -f /content/tgwebmobile_env.sh ]; then
  source /content/tgwebmobile_env.sh
else
  export JAVA_HOME=/usr/lib/jvm/java-17-openjdk-amd64
  export ANDROID_SDK_ROOT=/content/android-sdk
  export ANDROID_HOME=$ANDROID_SDK_ROOT
  export PATH=$JAVA_HOME/bin:$ANDROID_SDK_ROOT/platform-tools:$ANDROID_SDK_ROOT/cmdline-tools/latest/bin:$PATH
fi

echo "[BUILD] start $(date -u)"
cd /content/tgwebmobile
./gradlew --no-daemon :app:assembleDebug
echo "[BUILD] done $(date -u)"


In [ ]:
%%bash
set -euxo pipefail
APK=/content/tgwebmobile/app/build/outputs/apk/debug/app-debug.apk
test -f "$APK"
ls -lh "$APK"
sha256sum "$APK"
echo "Artifact verified: $APK"
